<a href="https://colab.research.google.com/github/CalculatedContent/ww-phys_theory/blob/master/FastJacobianDiag_Resnets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Generate different ResNet Fast Jacobian Diagonal data![alt text](https://)



```
resnet20_cifar10
resnet56_cifar10
resnet110_cifar10
resnet164bn_cifar10
resnet272bn_cifar10
resnet542bn_cifar10
resnet1001_cifar10
resnet1202_cifar10
```




In [24]:
Jdir =  '/content/drive/My Drive/resnets_fast'

thismodel = 'resnet20_cifar10'
inum_start = 0
batch_size = 100

time: 2.11 ms


In [25]:

!pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
time: 2.77 s


In [26]:
!pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.68 s


In [27]:
!pip install pytorchcv
!pip install powerlaw

time: 5.32 s


In [28]:
!pip install GPUtil
import GPUtil


GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
time: 2.8 s


In [29]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 1.7 ms


In [30]:
!mkdir "$Jdir"
!cd "$Jdir"; ls | grep $thismodel

mkdir: cannot create directory ‘/content/drive/My Drive/resnets_fast’: File exists
time: 1.51 s


In [31]:
!cd "$Jdir";pwd

/content/drive/My Drive/resnets_fast
time: 869 ms


In [32]:
!import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

import math
import copy




/bin/bash: import: command not found
time: 798 ms


In [33]:
def get_data(batch_size=100, train_range=None, test_range=None):
  normalize = transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]], std=[x / 255.0 for x in [63.0, 62.1, 66.7]])
  transform_train = transforms.Compose([
          transforms.ToTensor(),
          normalize])
  transform_test = transforms.Compose([
        transforms.ToTensor(),
        normalize])
  train_dataset = datasets.CIFAR10(root='data', 
                                train=True, 
                                transform=transform_train,
                                download=True)
  test_dataset = datasets.CIFAR10(root='data', 
                                train=False, 
                                transform=transform_test,
                                download=True)  
  
  if train_range:
      train_dataset = torch.utils.data.Subset(train_dataset, train_range)

  if test_range:
      teat_dataset = torch.utils.data.Subset(test_dataset, test_range)


  train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size,
                          num_workers=4,
                          shuffle=False)
  test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         num_workers=4,
                         shuffle=False)
  return train_dataset, test_dataset, train_loader, test_loader

time: 17.1 ms


### $J^{T}J$ Diagonal elements



In [39]:

from torch.autograd.gradcheck import zero_gradients

def fast_jacobian(inputs, output, batch_size, data_dim):
	"""
	input: input for the function for which the Jacobian will
	computed. It will be batch_size*data_dim. Make sure that the
	input is flagged as requires_grad=True with the torch.autograd.Variable
	wrapper. 
	output: output of the function for which the Jacobian will
	be computed. It will be batch_size*classes
	return: Jacobian of dimension batch_size*classes*data_dim
	"""
	assert inputs.requires_grad
	num_classes = output.size()[1] #0 index is batch

	jacobian = torch.zeros(num_classes, *inputs.size())
	grad_output = torch.zeros(*output.size())
	if inputs.is_cuda:
		grad_output = grad_output.cuda()
		jacobian = jacobian.cuda()

	#zero out gradients
	#compute gradient for one of the classes
	for i in range(num_classes):
		zero_gradients(inputs)
		grad_output.zero_()
		grad_output[:,i] = 1
		output.backward(grad_output, retain_graph=True)
		jacobian[i] = inputs.grad.data

	J =  torch.transpose(jacobian, dim0=0, dim1=1)
	J = J.reshape(batch_size, data_dim)
 
	return J


time: 13.1 ms


In [40]:
def jacobian_diagonal(model, data_loader, batch_size, num_classes=10, device='cuda:0', data_dim=3*32*32):
  '''compute J(J*v) diagnonal elements , where J is the jacobian,'''

  # compute Jdiag
  Jdiag = []
  model.eval()
  model = model.to(device)

  for batch, data in enumerate(data_loader):
    features, _ = data
    
    inputs = features.to(device)
    inputs.requires_grad=True
    outputs = model(inputs)

    J = fast_jacobian(inputs, outputs, batch_size, num_classes*data_dim)

    Jt = J.clone().transpose_(0,1)
    batch_diag = torch.mm(J,Jt).to('cpu')
    
    del J, Jt
    torch.cuda.empty_cache()

    for ib in range(batch_size):
      Jdiag.append(batch_diag[ib, ib].to('cpu').numpy())

    del batch_diag
    torch.cuda.empty_cache()

  return np.array(Jdiag)

time: 12.8 ms


### ResNet 

In [41]:
import pytorchcv
from pytorchcv.model_provider import get_model as ptcv_get_model

import powerlaw
for modelname in pytorchcv.model_provider._models.keys():
    if modelname.startswith('resnet') and modelname.endswith('cifar10'):
        print(modelname)

resnet20_cifar10
resnet56_cifar10
resnet110_cifar10
resnet164bn_cifar10
resnet272bn_cifar10
resnet542bn_cifar10
resnet1001_cifar10
resnet1202_cifar10
time: 3.48 ms


In [42]:
num_classes= 10
device = 'cuda:0'
num_data = 50000

time: 1.43 ms


In [0]:

model = ptcv_get_model(thismodel, pretrained=True)

for num in range(inum_start, 10):
  print("num {}".format(num))
  istart = num*5000
  iend = (num+1)*5000
  train_dataset, _, train_loader, _ = get_data(batch_size=batch_size, train_range=range(istart, iend))

  Jdiag = jacobian_diagonal(model, train_loader, batch_size)
  
  filename = "{}/Jdiag_fast_{}__{}.csv".format(Jdir, thismodel, num)
  np.savetxt(filename, Jdiag, delimiter='\t')


num 0
Files already downloaded and verified
Files already downloaded and verified
num 1
Files already downloaded and verified
Files already downloaded and verified
num 2
Files already downloaded and verified
Files already downloaded and verified
num 3
Files already downloaded and verified
Files already downloaded and verified
num 4
Files already downloaded and verified
Files already downloaded and verified
